In [2]:
import pandas as pd
import numpy as np
import pickle
from keras.models import load_model

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

In [3]:
train_ID_score = pd.read_excel("../CXR/train_test_split.xlsx", sheet_name = 'train')
test_ID_score = pd.read_excel("../CXR/train_test_split.xlsx", sheet_name = 'test')

train_ID_list = [int(i.split('_')[0]) for i in list(train_ID_score.ID)]
test_ID_list = [int(i.split('_')[0]) for i in list(test_ID_score.ID)]

In [4]:
### NTUH dataset
dataset = pd.read_csv('../concat_influ8_severity_score.csv')
dataset = dataset[['ID', 'Gender', 'Age', 'BMI', 'Diastolic_blood_pressure', 'Heart_rate', 
                   'Respiratory_rate', 'Creatinine', 'CRP', 'Hemoglobin', 'Bicarbonate', 
                   'PCO2', 'pH', 'Platelet', 'PO2',  'Blood_urea_nitrogen', 
                   'Lactic_acid', 'INR', 'Glucose', 'Hematocrit', 'severity_score', 'Survival_30']]
train = dataset.loc[dataset['ID'].isin(train_ID_list)]  #534筆(佔69.8%)
train = train.drop('ID', axis = 1)
test = dataset.loc[dataset['ID'].isin(test_ID_list)]  #231筆(佔30.2%)
test = test.drop('ID', axis = 1)

### random sampling do not have good results
# X_train = train.drop(['severity_score', 'Survival_30'], axis = True)
# X_test = test.drop(['severity_score', 'Survival_30'], axis = True)
# y_train = train[['Survival_30']]
# y_test = test[['Survival_30']]

### Resampling
count_class_0, count_class_1 = train.Survival_30.value_counts()
df_class_0 = train[train['Survival_30'] == 0]
df_class_1 = train[train['Survival_30'] == 1]
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

### validation ###
X_train = df_test_over.drop(['severity_score', 'Survival_30'], axis = 1)
y_train = df_test_over[['Survival_30']]
X_test = test.drop(['severity_score', 'Survival_30'], axis = 1)
y_test = test[['Survival_30']]

In [5]:
type(X_train)

,Gender,Age,BMI,Diastolic_blood_pressure,Heart_rate,Respiratory_rate,Creatinine,CRP,Hemoglobin,Bicarbonate,PCO2,pH,Platelet,PO2,Blood_urea_nitrogen,Lactic_acid,INR,Glucose,Hematocrit
2,0,84,23.309053,53.0,111.0,36.0,1.2,11.507600,13.5,21.100000,38.600000,7.341000,154.0,66.100,26.214345,1.4600,0.9100,97.00,38.1
3,1,51,18.179081,80.0,102.0,18.0,0.7,6.973314,9.3,24.177143,36.668857,7.412449,362.0,51.168,14.655567,1.7673,1.0000,140.00,26.8
4,0,76,27.162695,88.0,97.0,26.0,1.4,11.618143,12.4,19.600000,39.800000,7.297000,134.0,39.000,65.700000,3.7464,1.0300,561.00,30.7
5,1,35,24.977043,93.0,105.0,24.0,0.9,5.334771,14.4,23.200000,34.600000,7.430000,404.0,49.300,34.852000,2.6564,0.9470,122.16,41.5
6,1,85,22.884129,67.0,87.0,24.0,0.8,7.139314,10.2,23.000000,34.800000,7.425000,286.0,39.700,24.855300,2.0844,1.0094,40.00,33.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,0,87,23.875115,76.0,69.0,21.0,2.8,14.703057,8.9,21.300000,43.000000,7.298000,206.0,46.700,18.804000,2.3085,1.0220,184.00,25.5
284,0,75,24.730007,97.0,147.0,40.0,0.5,29.170000,11.7,21.900000,26.300000,7.524000,169.0,54.900,12.500000,1.7300,0.9700,159.00,30.9
656,0,76,24.679402,86.0,84.0,12.0,1.3,3.270000,10.2,40.800000,62.600000,7.418000,307.0,76.400,37.200000,0.9900,1.0800,228.00,33.0
247,0,66,21.554709,60.0,103.0,32.0,0.7,10.497714,14.0,21.200000,33.900000,7.401000,181.0,15.100,8.800000,3.3500,1.1100,123.00,33.3


In [4]:
X_train.shape

(1006, 19)

In [5]:
y_train.shape

(1006, 1)

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(16, activation="relu", input_shape=(19,)))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks = [keras.callbacks.EarlyStopping(monitor='accuracy', patience=8)])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

Epoch 1/100
26/26 [==============================] - 2s 73ms/step - loss: 23.3037 - accuracy: 0.6321 - val_loss: 26.9938 - val_accuracy: 0.0000e+00
Epoch 2/100
26/26 [==============================] - 0s 2ms/step - loss: 8.5003 - accuracy: 0.6071 - val_loss: 7.8167 - val_accuracy: 0.0000e+00
Epoch 3/100
26/26 [==============================] - 0s 2ms/step - loss: 2.0166 - accuracy: 0.6161 - val_loss: 1.8159 - val_accuracy: 0.3614
Epoch 4/100
26/26 [==============================] - 0s 2ms/step - loss: 0.7974 - accuracy: 0.5270 - val_loss: 1.0711 - val_accuracy: 0.6535
Epoch 5/100
26/26 [==============================] - 0s 2ms/step - loss: 0.7570 - accuracy: 0.4316 - val_loss: 0.8926 - val_accuracy: 0.7079
Epoch 6/100
26/26 [==============================] - 0s 2ms/step - loss: 0.7383 - accuracy: 0.3664 - val_loss: 0.7689 - val_accuracy: 0.8119
Epoch 7/100
26/26 [==============================] - 0s 2ms/step - loss: 0.7204 - accuracy: 0.3771 - val_loss: 0.7292 - val_accuracy: 0.8713
Ep

In [7]:
model.save('joint_fusion_structure.hdf5')